In [1]:
from tools.numerical_gradient import *
from models.layers import *
import numpy as np

# Let's try out some simple functions for numerical_gradient. #

We know the linear equation y = 3x should always return 3. Let's check it:

In [2]:
def linear(x, slope=3):
    return slope*x

slope = numerical_gradient_check_scalar(linear, 5)
print slope

2.99999999989


In [3]:
def multi_quadratic(x):
    return x[0]**2 + x[1]
arr = np.array([2,2], float)

slope = numerical_gradient_check_multivar(multi_quadratic, arr)
print slope

[ 3.9999999   0.99999997]


In [4]:
def multi_cubic_field(x):
    return np.array([x[0]**3 + x[1]**2, x[0]*2 + x[1]/12])
arr = np.array([3,4], dtype=np.float32)

def matrix_mult(x, b = np.array([[3,5],[2,1]])):
    return x.dot(b)
                
vector_field = numerical_gradient_check_multivar(multi_cubic_field, arr)
print vector_field

arr = np.array([[3,4],[1,2]], dtype=np.float32)
vector_field = numerical_gradient_check_multivar(matrix_mult, arr)
print vector_field

[ 29.03938293   8.09431076]
[[ 8.01086426  3.0040741 ]
 [ 8.01086426  3.0040741 ]]


In [5]:
def affine_transform(w, x, b):
    return x.dot(w) + b

x = np.array([[1,2]], float) # 1 x 2
w = np.array([[3,2,1],[1,2,5]], float) # 2 x 3
b = np.array([[1,5,7]], float) # 1 x 3

fw = lambda w: affine_transform(w,x,b)
fx = lambda x: affine_transform(w,x,b)
fb = lambda b: affine_transform(w,x,b)

vector_field = numerical_gradient_check_multivar(fw, w)
print "fw : ", vector_field
vector_field = numerical_gradient_check_multivar(fx, x)
print "fx : ", vector_field
vector_field = numerical_gradient_check_multivar(fb, b)
print "fb : ", vector_field

fw :  [[ 0.99999997  0.99999997  0.99999997]
 [ 1.99999995  1.99999995  1.99999995]]
fx :  [[ 5.99999985  7.9999998 ]]
fb :  [[ 0.99999997  0.99999997  0.99999997]]


# word_embedding_forward/backward #

In [6]:
# Looks good to me
ans = np.array([[[3, 4, 7, 1],
                [3, 4, 7, 1],
                [1, 5, 9, 4]],

               [[1, 5, 9, 4],
                [1, 5, 9, 4],
                [1, 5, 9, 4]],

               [[4, 3, 2, 5],
                [3, 4, 7, 1],
                [4, 3, 2, 5]]])

x = np.array([[1,1,0], [0,0,0], [2,1,2]], int)
words = np.array([[1,5,9,4],[3,4,7,1],[4,3,2,5]])
arr = word_embedding_forward(words, x)

assert np.array_equal(ans, arr)
print arr.shape, "\n", arr

(3, 3, 4) 
[[[3 4 7 1]
  [3 4 7 1]
  [1 5 9 4]]

 [[1 5 9 4]
  [1 5 9 4]
  [1 5 9 4]]

 [[4 3 2 5]
  [3 4 7 1]
  [4 3 2 5]]]


In [8]:
dout = np.array([[[1,2,0,1],[3,2,9,1],[1,2,1,1]],
                 [[3,9,2,4],[1,9,9,0],[2,0,1,6]],
                 [[1,0,1,0],[0,1,0,5],[3,0,0,1]]])

arr = word_embedding_backward(dout, words, x)
ans = np.array([[  7.,  20.,  13.,  11.],
               [  4.,   5.,   9.,   7.],
               [  4.,   0.,   1.,   1.]])

assert np.array_equal(ans, arr)
print arr.shape, "\n", arr

(3, 4) 
[[  7.  20.  13.  11.]
 [  4.   5.   9.   7.]
 [  4.   0.   1.   1.]]


# Tanh Vanilla RNN_step Layer #

In [2]:
# Forward
"""
prev_h = (N,H)
x = (N, V)
W_hh = (H,H)
W_xh = (V,H)
b = (H,)
"""
prev_h = np.random.random((3,5)) # N = 3, H = 5
x = np.random.random((3,4)) # N = 3, V = 4
W_hh = np.random.random((5,5)) # H = 5
W_xh = np.random.random((4,5)) # V = 4, H = 5
b = np.random.random((5,)) # H = 5

res = rnn_step_forward(prev_h, W_hh, x, W_xh, b) # N = 3, H = 5
print res.shape

(3, 5)


In [3]:
# Backward
fprev_h = lambda prev_h: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fW_hh = lambda W_hh: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fx = lambda x: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fW_xh = lambda W_xh: rnn_step_forward(prev_h, W_hh, x, W_xh, b)
fb = lambda b: rnn_step_forward(prev_h, W_hh, x, W_xh, b)

dprev_h_num = numerical_gradient_check_multivar(fprev_h, prev_h)
dW_hh_num = numerical_gradient_check_multivar(fW_hh, W_hh)
dx_num = numerical_gradient_check_multivar(fx, x)
dW_xh_num = numerical_gradient_check_multivar(fW_xh, W_xh)
db_num = numerical_gradient_check_multivar(fb, b)

dW_hh, dW_xh, dprev_h, dx, db = rnn_step_backward(prev_h, W_hh, x, W_xh, b, np.ones_like(res))
print "dprev_h : ", norm_loss(dprev_h, dprev_h_num)
print "dW_hh : ", norm_loss(dW_hh, dW_hh_num)
print "dx : ", norm_loss(dx, dx_num)
print "dW_xh : ", norm_loss(dW_xh, dW_xh_num)
print "db : ", norm_loss(db, db_num)

dprev_h :  1.26232714804e-08
dW_hh :  1.26096801212e-08
dx :  1.26003102835e-08
dW_xh :  1.26312391781e-08
db :  1.25970797693e-08


# Tanh Vanilla RNN Layer #

In [2]:
# Forward
from tools.numerical_gradient import *
from models.layers import *
import numpy as np
"""
h0 = (N,H)
W_hh = (H,H)
x = (N,T,D)
W_xh = (D,H)
b = (H,)
"""
N = 3
D = 4
H = 5
T = 1

h0 = np.random.random((N,H))
W_hh = np.random.random((H,H))
x = np.random.random((N,T,D))
W_xh = np.random.random((D,H))
b = np.random.random((H,))

h = rnn_forward(x, W_xh, W_hh, b, h0)

print h.shape

(3, 1, 5)


models/layers.py:161: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if h0 != None: # Supply an h0 state.


In [3]:
# Backward
fx = lambda x: rnn_forward(x, W_xh, W_hh, b, h0)
fW_xh = lambda W_xh: rnn_forward(x, W_xh, W_hh, b, h0)
fW_hh = lambda W_hh: rnn_forward(x, W_xh, W_hh, b, h0)
fb = lambda b: rnn_forward(x, W_xh, W_hh, b, h0)
fh0 = lambda h0: rnn_forward(x, W_xh, W_hh, b, h0)

dx_num = numerical_gradient_check_multivar(fx, x)
dW_xh_num = numerical_gradient_check_multivar(fW_xh, W_xh)
dW_hh_num = numerical_gradient_check_multivar(fW_hh, W_hh)
db_num = numerical_gradient_check_multivar(fb, b)
dh0_num = numerical_gradient_check_multivar(fh0, h0)

dW_hh, dW_xh, dx, db, dh0 = rnn_backward(x, W_xh, W_hh, b, h0, h, np.ones_like(h))
print "dW_hh : ", norm_loss(dW_hh, dW_hh_num)
print "dW_xh : ", norm_loss(dW_xh, dW_xh_num)
print "dx : ", norm_loss(dx, dx_num)
print "db : ", norm_loss(db, db_num)
print "dh0 : ", norm_loss(dh0, dh0_num)

dW_hh :  1.26022016596e-08
dW_xh :  1.26066582029e-08
dx :  1.26172189348e-08
db :  1.26088284757e-08
dh0 :  1.26056472474e-08


# Affine Layer #

In [11]:
# Forward
"""
h = (N,H)
W_hy = (H,D)
b = (D,)
"""
h = np.random.random((3,5)) # N = 3, H = 5
W_hy = np.random.random((5,7)) # H = 5, D = 7
b = np.random.random((7,)) # D = 7

res = affine_forward(h, W_hy, b) # N = 3, D = 7
print res.shape

(3, 7)


In [15]:
fh = lambda h: affine_forward(h, W_hy, b)
fW_hy = lambda W_hy: affine_forward(h, W_hy, b)
fb = lambda b: affine_forward(h, W_hy, b)

dh_num = numerical_gradient_check_multivar(fh, h)
dW_hy_num = numerical_gradient_check_multivar(fW_hy, W_hy)
db_num = numerical_gradient_check_multivar(fb, b)

dh, dW_hy, db = affine_backward(h, W_hy, b, np.ones_like(res))

print "dx : ", norm_loss(dh, dh_num)
print "dW_xh : ", norm_loss(dW_hy, dW_hy_num)
print "db : ", norm_loss(db, db_num)

dx :  1.26304513777e-08
dW_xh :  1.26317338729e-08
db :  1.26310626518e-08


# Softmax Layer - One of the most important functions in Deep Learning #

In [13]:
# Forward and backwards
"""
x = (N,D)
y = (N,)
"""
x = np.random.random((3,4)) # N = 3, D = 4
y = np.random.randint(4, size=3) # D = 4, N = 3

fx = lambda x: softmax(x, y)[0]

loss, dJ = softmax(x, y)
dJ_num = numerical_gradient_check_multivar(fx, x)
print "dJ : ", norm_loss(dJ, dJ_num)

dJ :  1.26316073922e-08


# SVM Layer - The other most important functions in Deep Learning #

In [14]:
# Forward and backwards
"""
x = (N,D)
y = (N,)
"""
x = np.random.random((3,5)) # N = 3, D = 5
y = np.random.randint(5, size=3) # D = 5, N = 3

fx = lambda x: SVM(x, y)[0]

loss, dJ = SVM(x, y)
dJ_num = numerical_gradient_check_multivar(fx, x)
print "dJ : ", norm_loss(dJ, dJ_num)

dJ :  1.26399460897e-08


# The entire RNN forward() #